## 서울 부동산 실거래가 데이터(from 공공데이터포털)
URL: https://data.seoul.go.kr/dataList/OA-21275/S/1/datasetView.do

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.getOrCreate()

24/03/20 14:16:24 WARN Utils: Your hostname, dead-line.local resolves to a loopback address: 127.0.0.1; using 172.29.89.178 instead (on interface en0)
24/03/20 14:16:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/20 14:16:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/20 14:16:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
raw_df = spark.read.format('json').load('../data/real-estate-seoul-gangnamgu.json')

In [4]:
raw_df.show()

+--------------------+--------+---------+---------+---------+-----------------------------+-------+------+----------+--------+--------+-----+----------+--------+-----------+-------+--------------+--------+---------+------+------+--------+
|     _corrupt_record|acc_year|bjdong_cd|bjdong_nm|bldg_area|                      bldg_nm|bonbeon|bubeon|build_year|cntl_ymd|deal_ymd|floor|house_type|land_gbn|land_gbn_nm|obj_amt|rdealer_lawdnm| req_gbn|right_gbn|sgg_cd|sgg_nm|tot_area|
+--------------------+--------+---------+---------+---------+-----------------------------+-------+------+----------+--------+--------+-----+----------+--------+-----------+-------+--------------+--------+---------+------+------+--------+
|                   {|    NULL|     NULL|     NULL|     NULL|                         NULL|   NULL|  NULL|      NULL|    NULL|    NULL| NULL|      NULL|    NULL|       NULL|   NULL|          NULL|    NULL|     NULL|  NULL|  NULL|    NULL|
|"DESCRIPTION" : {...|    NULL|     NULL|   

In [5]:
filter_df = raw_df.filter(col('acc_year') != 'NULL')

In [18]:
(filter_df.write.format('parquet')
    .mode('overwrite')
    .save('../data/real-estate-seoul-gangnamgu')
)


24/03/20 14:12:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/20 14:12:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/20 14:12:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/03/20 14:12:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/03/20 14:12:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/03/20 14:12:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/03/20 14:12:31 WARN MemoryManager: Total allocation exceeds 95.

## Load parquet files with no projection

In [13]:
parquet_df = spark.read.format('parquet').load('../data/real-estate-seoul-gangnamgu/*')
parquet_df.persist()
parquet_df = parquet_df.select('acc_year', 'bjdong_cd', 'bjdong_nm', 'bldg_area')

parquet_df.explain()

parquet_df.show()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- InMemoryTableScan [acc_year#519, bjdong_cd#520, bjdong_nm#521, bldg_area#522]
      +- InMemoryRelation [_corrupt_record#518, acc_year#519, bjdong_cd#520, bjdong_nm#521, bldg_area#522, bldg_nm#523, bonbeon#524, bubeon#525, build_year#526, cntl_ymd#527, deal_ymd#528, floor#529L, house_type#530, land_gbn#531, land_gbn_nm#532, obj_amt#533, rdealer_lawdnm#534, req_gbn#535, right_gbn#536, sgg_cd#537, sgg_nm#538, tot_area#539], StorageLevel(disk, memory, deserialized, 1 replicas)
            +- *(1) ColumnarToRow
               +- FileScan parquet [_corrupt_record#518,acc_year#519,bjdong_cd#520,bjdong_nm#521,bldg_area#522,bldg_nm#523,bonbeon#524,bubeon#525,build_year#526,cntl_ymd#527,deal_ymd#528,floor#529L,house_type#530,land_gbn#531,land_gbn_nm#532,obj_amt#533,rdealer_lawdnm#534,req_gbn#535,right_gbn#536,sgg_cd#537,sgg_nm#538,tot_area#539] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(13 paths)[file:/

+--------+---------+---------+---------+
|acc_year|bjdong_cd|bjdong_nm|bldg_area|
+--------+---------+---------+---------+
|    2024|    10300|   개포동|    37.43|
|    2024|    10600|   대치동|    34.32|
|    2024|    10300|   개포동|     34.8|
|    2024|    10100|   역삼동|    14.77|
|    2024|    10600|   대치동|    77.98|
|    2024|    11200|   자곡동|     58.7|
|    2024|    10100|   역삼동|   627.91|
|    2024|    10100|   역삼동|     23.9|
|    2024|    10800|   논현동|    30.03|
|    2024|    10300|   개포동|    37.43|
|    2024|    10600|   대치동|    76.79|
|    2024|    10800|   논현동|    14.37|
|    2024|    10100|   역삼동|     20.7|
|    2024|    11500|   수서동|    97.63|
|    2024|    10800|   논현동|    29.79|
|    2024|    10600|   대치동|    94.49|
|    2024|    10300|   개포동|    84.36|
|    2024|    10600|   대치동|    17.68|
|    2024|    10100|   역삼동|    29.88|
|    2024|    11800|   도곡동|   119.89|
+--------+---------+---------+---------+


In [9]:
parquet_df = spark.read.format('parquet').load('../data/real-estate-seoul-gangnamgu/*')

parquet_df = parquet_df.select('acc_year', 'bjdong_cd', 'bjdong_nm', 'bldg_area')

parquet_df.explain()

parquet_df.show()

+---------------+--------+---------+---------+---------+-----------------------------+-------+------+----------+--------+--------+-----+----------+--------+-----------+-------+--------------+--------+---------+------+------+--------+
|_corrupt_record|acc_year|bjdong_cd|bjdong_nm|bldg_area|                      bldg_nm|bonbeon|bubeon|build_year|cntl_ymd|deal_ymd|floor|house_type|land_gbn|land_gbn_nm|obj_amt|rdealer_lawdnm| req_gbn|right_gbn|sgg_cd|sgg_nm|tot_area|
+---------------+--------+---------+---------+---------+-----------------------------+-------+------+----------+--------+--------+-----+----------+--------+-----------+-------+--------------+--------+---------+------+------+--------+
|           NULL|    2024|    10300|   개포동|    37.43|                예가풍경채3차|   1224|  0004|      2019|    NULL|20240315|    3|연립다세대|       1|       대지|  61000|   서울 용산구|중개거래|     NULL| 11680|강남구|    22.0|
|           NULL|    2024|    10600|   대치동|    34.32|         선릉역 롯데골드로즈2|   0890|  0059|   